In [240]:
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from itertools import chain
from pyspark.sql.functions import udf

In [40]:
DATAFOLDER = '/Users/christian/Data/udacity_capstone/'

In [130]:
def get_immi_schema():
    """
    Map the column names to datatypes and return as a schema.
    """
    immi_schema = T.StructType([
        T.StructField('cicid', T.IntegerType()),
        T.StructField('i94yr', T.StringType()),
        T.StructField('i94mon', T.StringType()),
        T.StructField('i94cit', T.StringType()),
        T.StructField('i94res', T.IntegerType()),
        T.StructField('i94port', T.StringType()),
        T.StructField('arrdate', T.StringType()),
        T.StructField('i94mode', T.StringType()),
        T.StructField('i94addr', T.StringType()),
        T.StructField('depdate', T.StringType()),
        T.StructField('i94bir', T.StringType()),
        T.StructField('i94visa', T.StringType()),
        T.StructField('count',  T.IntegerType()),
        T.StructField('dtadfile', T.StringType()),
        T.StructField('visapost', T.IntegerType()),
        T.StructField('occup', T.StringType()),
        T.StructField('entdepa', T.StringType()),
        T.StructField('entdepd', T.IntegerType()),
        T.StructField('entdepu', T.StringType()),
        T.StructField('matflag', T.StringType()),
        T.StructField('biryear', T.StringType()),
        T.StructField('dtaddto', T.StringType()),
        T.StructField('gender', T.StringType()),
        T.StructField('insnum', T.StringType()),
        T.StructField('airline', T.StringType()),
        T.StructField('admnum', T.StringType()),
        T.StructField('fltno', T.StringType()),
        T.StructField('visatype', T.StringType()),
    ])
    return immi_schema


In [131]:
schema = get_immi_schema()


In [138]:

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_spark = spark.read.option("mergeSchema", "true").parquet(os.path.join(DATAFOLDER, 'sas_data'))

# convert data types

In [149]:
df_spark = df_spark.withColumn('i94yr', df_spark['i94yr'].cast(T.IntegerType())).\
        withColumn('i94mon', df_spark['i94mon'].cast(T.IntegerType())).\
        withColumn('i94cit', df_spark['i94cit'].cast(T.IntegerType())).\
        withColumn('i94res', df_spark['i94res'].cast(T.IntegerType())).\
        withColumn('arrdate', df_spark['arrdate'].cast(T.IntegerType())).\
        withColumn('i94mode', df_spark['i94mode'].cast(T.IntegerType())).\
        withColumn('depdate', df_spark['depdate'].cast(T.IntegerType())).\
        withColumn('i94bir', df_spark['i94bir'].cast(T.IntegerType())).\
        withColumn('i94visa', df_spark['i94visa'].cast(T.IntegerType())).\
        withColumn('count', df_spark['count'].cast(T.IntegerType())).\
        withColumn('biryear', df_spark['biryear'].cast(T.IntegerType())).\
        withColumn('admnum', df_spark['admnum'].cast(T.IntegerType()))

In [112]:
df_sample = pd.read_csv(os.path.join(DATAFOLDER, 'immigration_data_sample.csv'))

In [150]:
df_spark.head()

Row(cicid=459651.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20559, i94bir=54, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='O', entdepd='R', entdepu=None, matflag='M', biryear=1962, dtaddto='07012016', gender=None, insnum=None, airline='VS', admnum=2147483647, fltno='00115', visatype='WT')

In [151]:
df_time = df_spark.withColumn("arrival_date", F.expr("date_add(to_date('1960-01-01'), arrdate)"))

In [152]:
df_time.head()

Row(cicid=459651.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20559, i94bir=54, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='O', entdepd='R', entdepu=None, matflag='M', biryear=1962, dtaddto='07012016', gender=None, insnum=None, airline='VS', admnum=2147483647, fltno='00115', visatype='WT', arrival_date=datetime.date(2016, 4, 3))

In [153]:
df_time.show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|arrival_date|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+
|459651.0| 2016|     4|   135|   135|    ATL|  20547|      1|     FL|  20559|    54|      2|    1|20160403|    null| null|      O|      R|   null|      M|   1962|07012016|  null|  null|     VS|2147483647|00115|      WT|  2016-04-03|
|459652.0| 2016|     4|   135|   135|    ATL|  20547|      1|     FL

## map i94addrl - state

In [225]:

def map_col(df, map_col_name, df_col_name, new_col_name):
    """
    
    Parameters
    ----------
    df : spark dataframe
        The file containing the df_col_name to be used for mapping.
    map_col_name : str
        The column name of the mapping file.
    df_col_name : str
        The column name in the Spark dataframe to be used.
    new_col_name : str
        New column name of the mapping results.
    
    
    """
    df_map = pd.read_csv(os.path.join(DATAFOLDER, f'{map_col_name}.csv'), quotechar="'")
    dic_map = dict(zip(df_map['value'], df_map[map_col_name]))
    mapping_expr = F.create_map([F.lit(x) for x in chain(*dic_map.items())])
    return df.withColumn(new_col_name, mapping_expr[F.col(df_col_name)])
    

In [160]:
addrl = pd.read_csv(os.path.join(DATAFOLDER, 'i94addrl.csv'))

In [81]:
colname = addrl.columns[1]
addrl_dic = dict(zip(addrl['value'], addrl[colname]))

In [90]:
mapping_expr = F.create_map([F.lit(x) for x in chain(*addrl_dic.items())])

In [100]:
df_time.withColumn('state', mapping_expr[F.col('i94addr')]).show()

+--------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+------------+-------+
|   cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|arrival_date|  state|
+--------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+------------+-------+
|459651.0|2016.0|   4.0| 135.0| 135.0|    ATL|20547.0|    1.0|     FL|20559.0|  54.0|    2.0|  1.0|20160403|    null| null|      O|      R|   null|      M| 1962.0|07012016|  null|  null|     VS|5.5556253633E10|00115|      WT|  2016-04-03|FLORIDA|
|459652.0|20

In [103]:
!ls /Users/christian/Data/udacity_capstone/

I94_SAS_Labels_Descriptions.SAS
airport-codes_csv.csv
climate-change-earth-surface-temperature-data
climate-change-earth-surface-temperature-data.zip
i94addrl.csv
i94cntyl.csv
i94cntyl.sas
i94model.csv
i94prtl.csv
i94visa.csv
immigration_data_sample.csv
sas_data
us-cities-demographics.csv


In [215]:
df_test = df_time
map_col_name = 'i94cntyl'
df_col_name = 'i94cit'
new_col_name = 'country_cit'
df_map = pd.read_csv(os.path.join(DATAFOLDER, f'{map_col_name}.csv'), quotechar="'")
dic_map = dict(zip(df_map['value'], df_map[map_col_name]))
mapping_expr = F.create_map([F.lit(x) for x in chain(*dic_map.items())])
df_test_2 = df_test.withColumn(new_col_name, mapping_expr[F.col(df_col_name)])

In [222]:
df_map

,value,i94cntyl
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
...,...,...
284,791,No Country Code (791)
285,849,No Country Code (849)
286,914,No Country Code (914)
287,944,No Country Code (944)


In [235]:
#df, map_col_name, df_col_name, new_col_name
df_test = df_time
df_test = map_col(df_test, 'i94cntyl', 'i94cit', 'state_cit')
df_test = map_col(df_test, 'i94cntyl', 'i94res', 'state_res')
df_test = map_col(df_test, 'i94prtl', 'i94port', 'port')
df_test = map_col(df_test, 'i94addrl', 'i94addr', 'us_state')
df_test = map_col(df_test, 'i94visa', 'i94visa', 'visa')

In [236]:
df_test.show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+--------------+--------------+--------------------+--------+--------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|arrival_date|     state_cit|     state_res|                port|us_state|    visa|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+--------------+--------------+--------------------+--------+--------+
|459651.0| 2016|     4|   135|   135|    ATL|  20547|      1|     FL|  20559|    54|      2|  

In [239]:
df_test.head(10)

[Row(cicid=459651.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20559, i94bir=54, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='O', entdepd='R', entdepu=None, matflag='M', biryear=1962, dtaddto='07012016', gender=None, insnum=None, airline='VS', admnum=2147483647, fltno='00115', visatype='WT', arrival_date=datetime.date(2016, 4, 3), state_cit='UNITED KINGDOM', state_res='UNITED KINGDOM', port='ATLANTA, GA           ', us_state='FLORIDA', visa='Pleasure'),
 Row(cicid=459652.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20555, i94bir=74, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='T', entdepd='O', entdepu=None, matflag='M', biryear=1942, dtaddto='07012016', gender='F', insnum=None, airline='VS', admnum=674406485, fltno='103', visatype='WT', arrival_date=datetime.date(2016, 4, 3), state_cit='UNIT

In [168]:
df_test

# clean up city name of airport

In [247]:
@udf
def udf_city_name(city_full):
    return str.split(city_full, ',')[0].capitalize()

In [248]:
df_test = df_test.withColumn("port_city", udf_city_name("port"))

In [250]:
df_test.head()

Row(cicid=459651.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20559, i94bir=54, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='O', entdepd='R', entdepu=None, matflag='M', biryear=1962, dtaddto='07012016', gender=None, insnum=None, airline='VS', admnum=2147483647, fltno='00115', visatype='WT', arrival_date=datetime.date(2016, 4, 3), state_cit='UNITED KINGDOM', state_res='UNITED KINGDOM', port='ATLANTA, GA           ', us_state='FLORIDA', visa='Pleasure', port_city='Atlanta')

# merge with demographic data